### Predict Test

- firesmoke_detect_tuned2
- firesmoke_yolov8s_tuned
- firesmoke_seg_train

#### 단일 이미지 예측

In [6]:
from ultralytics import YOLO
import time

# 1. 모델 로드
model_path = "runs/firesmoke_detect_tuned2/weights/best.pt"  # 실험별로 경로 수정
model = YOLO(model_path)

# 2. 예측할 이미지 경로
source = "imgs/naroho.jpg"

# 3. 추론 시간 측정 시작
start = time.time()

# 4. 예측 수행
results = model.predict(
    source=source,
    save=True,              # 예측 결과 이미지 저장
    save_txt=True,          # 예측 박스 또는 마스크 좌표 저장
    imgsz=768,              # 실험에 맞게 설정
    conf=0.25,              # confidence threshold
    iou=0.5,                # IoU threshold
    device=0,               # GPU 사용
    project="predictions",  # 결과 저장 위치
    name="exp4_test"        # 실험별로 구분
)

# 5. 추론 시간 측정 종료
end = time.time()
print(f"추론 시간: {end - start:.3f}초")



image 1/1 c:\Users\main\workspace\vision\project\imgs\naroho.jpg: 768x512 1 fire, 1 smoke, 24.1ms
Speed: 4.6ms preprocess, 24.1ms inference, 2.1ms postprocess per image at shape (1, 3, 768, 512)
Results saved to predictions\exp4_test
1 label saved to predictions\exp4_test\labels
추론 시간: 0.139초


### OpenCV 웹캠 실시간 추론 + FPS 측정 + 평균 추론 시간
model = YOLO("runs/firesmoke_detect_tuned2/weights/best.pt")
# - firesmoke_detect_tuned2
# - firesmoke_yolov8s_tuned
# - firesmoke_seg_train

In [2]:
import cv2

for i in range(5):
    cap = cv2.VideoCapture(i)
    if cap.isOpened():
        print(f"✅ 웹캠 {i} 번 열림")
        cap.release()
    else:
        print(f"❌ 웹캠 {i} 번 안 열림")


✅ 웹캠 0 번 열림
❌ 웹캠 1 번 안 열림
❌ 웹캠 2 번 안 열림
❌ 웹캠 3 번 안 열림
❌ 웹캠 4 번 안 열림


In [12]:
import cv2
import time
from ultralytics import YOLO

# YOLO 모델 불러오기
model = YOLO("runs/firesmoke_detect_tuned2/weights/best.pt")

# 웹캠 열기
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ 웹캠 열기 실패")
    exit()

# 해상도 설정 (선택)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

print("✅ 웹캠 실행 중... 'q' 키를 누르면 종료합니다.")

# 프레임 처리
while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ 프레임 읽기 실패")
        break

    # YOLO 추론
    results = model(frame, conf=0.6, verbose=False)[0]
    annotated_frame = results.plot()

    # FPS 텍스트 추가
    fps = cap.get(cv2.CAP_PROP_FPS)
    cv2.putText(annotated_frame, f"FPS: {fps:.2f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # 실제 화면에 출력
    cv2.imshow("🔥 YOLO 실시간 탐지", annotated_frame)

    # 'q' 키 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


✅ 웹캠 실행 중... 'q' 키를 누르면 종료합니다.


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [14]:
import streamlit as st
import cv2
from ultralytics import YOLO

st.title("🔥 YOLOv8 실시간 화재/연기 탐지")
run = st.checkbox("▶️ 웹캠 실행")
FRAME_WINDOW = st.image([])

# YOLO 모델 로드
model = YOLO("runs/firesmoke_detect_tuned2/weights/best.pt")
cap = cv2.VideoCapture(0)

while run:
    ret, frame = cap.read()
    if not ret:
        st.warning("❌ 프레임 읽기 실패")
        break

    # 추론 수행
    results = model(frame, conf=0.6, iou=0.5, verbose=False)[0]
    annotated = results.plot()

    # Streamlit에 프레임 출력
    FRAME_WINDOW.image(annotated, channels="BGR")

cap.release()


2025-06-12 16:07:21.301 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 16:07:21.361 
  command:

    streamlit run c:\Users\main\miniconda3\envs\dl\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-06-12 16:07:21.362 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 16:07:21.362 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 16:07:21.363 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 16:07:21.363 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 16:07:21.364 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 16:07:21.365 Thread 'MainThr